In [ ]:
from kafka3 import KafkaConsumer
from json import loads
import pandas as pd
import networkx as nx
from IPython.display import clear_output
import time

hostip = "192.168.1.111"

class TransactionGraphUpdater:
    def __init__(self):
        self.transaction_graph = nx.Graph()
        self.edge_amounts = {}
        self.last_update_time = time.time()  # Store the last update time

    def _update_plot(self):
        current_time = time.time()
        # Check if enough time has passed since the last update
        if current_time - self.last_update_time >= 10: 
            clear_output(wait=True)
            print("Node count:", len(self.transaction_graph.nodes))
            print("Edge count:", len(self.transaction_graph.edges))
            self.last_update_time = current_time

    def update_transaction_graph(self, df):
        for index, row in df.iterrows():
            from_account = row.get('Account')
            to_account = row.get('To_Account')
            amount = row.get('Amount_Received')
            if from_account is not None and to_account is not None:
                edge = (from_account, to_account)
                if self.transaction_graph.has_edge(*edge):
                    # Update the edge with the new amount
                    self.edge_amounts[edge] += amount
                else:
                    # Add a new edge
                    self.transaction_graph.add_edge(*edge)
                    self.edge_amounts[edge] = amount

        # Print the nodes and edges, and update the plot
        print("Node count:", len(self.transaction_graph.nodes))
        print("Edge count:", len(self.transaction_graph.edges))
        self._update_plot()

def consume_messages(consumer_instance, topic_name, graph_updater):
    try:
        consumer_instance.subscribe([topic_name])

        for message in consumer_instance:
            if message is not None:
                records = message.value
                preprocess_and_plot(records, graph_updater)

    except Exception as ex:
        print('Exception in consuming messages.')
        print(str(ex))

def preprocess_and_plot(data, graph_updater):
    if not isinstance(data, list) or not all(isinstance(record, dict) for record in data):
        print("Invalid data format. Expecting a list of dictionaries.")
        return

    df = pd.DataFrame(data)
    df_updated = create_features(df)

    # Update the transaction graph using the TransactionGraphUpdater instance
    graph_updater.update_transaction_graph(df_updated)

def connect_kafka_consumer():
    _consumer = None
    try:
        _consumer = KafkaConsumer(
            bootstrap_servers=[f'{hostip}:9092'],
            auto_offset_reset='earliest',
            enable_auto_commit=True,
            group_id='my-group',
            value_deserializer=lambda x: loads(x),
            api_version=(0, 10)
        )
    except Exception as ex:
        print('Exception while connecting Kafka consumer.')
        print(str(ex))
    finally:
        return _consumer

def create_features(df):
    if 'Transaction_Direction' not in df.columns:
        df['Transaction_Direction'] = df.apply(lambda row: 'Incoming' if row.get('Received_Amount', 0) > 0 else 'Outgoing', axis=1)

    if 'Currency_Conversion' not in df.columns:
        df['Currency_Conversion'] = df.get('Receiving_Currency') != df.get('Payment_Currency')

    if 'Amount_Ratio' not in df.columns:
        df['Amount_Ratio'] = df.apply(lambda row: float(row.get('Received_Amount', 0)) / float(row.get('Amount_Paid', 0)) if row.get('Received_Amount', 0) and row.get('Amount_Paid', 0) and float(row['Amount_Paid']) != 0 else 0, axis=1)

    return df

if __name__ == '__main__':
    topic = 'Transaction'
    print('Consuming records..')
    consumer = connect_kafka_consumer()

    if consumer is None:
        print('Exiting script due to connection error.')
        exit()

    updater = TransactionGraphUpdater()  # Initialize only once

    try:
        consumer.subscribe([topic])
        for message in consumer:
            if message is not None:
                records = message.value
                preprocess_and_plot(records, updater)

    except Exception as ex:
        print('Exception in consuming messages.')
        print(str(ex))


Consuming records..
Node count: 10
Edge count: 5
Node count: 15
Edge count: 10
Node count: 19
Edge count: 14
